In [1]:
with open("inputs/Day_22.txt") as f:
   input_data = f.read()

In [91]:
def part_1_solution(raw_input, deck_size):
    commands = raw_input.split("\n")
    
    initial_deck = list(range(deck_size))
    
    deck = shuffle(initial_deck, commands)
    
    for i, card in enumerate(deck):
        if card == 2019:
            return i

    
def shuffle(deck, commands):
    for command in commands:
        if command.startswith("deal into new stack"):
            deck.reverse()
        elif command.startswith("deal with increment"):
            interval = int(command[19:])
            deck = deal_with_increment(deck, interval)
        elif command.startswith("cut"):
            size = int(command[3:])
            deck = cut(deck, size)
        else:
            raise Exception("Wrong command")
        
    return deck

    
def deal_with_increment(deck, interval):
    new_deck = [-1] * len(deck)
    
    current_idx = 0
    
    while deck:
        current_idx %= len(new_deck)
        card = deck.pop(0)
        new_deck[current_idx] = card
        
        current_idx += interval
    
    return new_deck


def cut(deck, size):
    if size >= 0:
        cut_portion = deck[:size]
        remaning_portion = deck[size:]
        new_deck = remaning_portion + cut_portion
    else:
        cut_portion = deck[size:]
        remaning_portion = deck[:size]
        new_deck = cut_portion +  remaning_portion
    
    return new_deck

In [93]:
print(f"Part 1 solution: {part_1_solution(input_data, 10007)}")

Part 1 solution: 6417


In [99]:
import sys
from functools import partial, reduce

# get modular multiplicative inverse for prime p
def prime_modinv(p, mod):
    return pow(p, mod - 2, mod)

# get (a, b) coefficients for inverse linear index mapping functions ax+b
def inverse_functions(instructions, ncards):
    for line in reversed(instructions):
        if line == 'deal into new stack':
            yield -1, ncards - 1
        elif line.startswith('cut'):
            amount = int(line[3:])
            yield 1, amount
        else:
            increment = int(line[19:])
            yield prime_modinv(increment, ncards), 0

# compose all inverse mappings into a single linear function
def inverse_function(instructions, ncards):
    functions = inverse_functions(instructions, ncards)
    return reduce(partial(fcompose, ncards), functions, (1, 0))

# compose linear functions ax+b and cx+d
def fcompose(mod, f, g):
    a, b = f
    c, d = g
    return c * a % mod, (c * b + d) % mod

# compute f(x) = ax+b
def fapply(f, x, mod):
    a, b = f
    return (a * x + b) % mod

# repeat ax+b n times
def frepeat(f, n, mod):
    if n == 0:
        return 1, 0
    if n == 1:
        return f
    half, odd = divmod(n, 2)
    g = frepeat(f, half, mod)
    gg = fcompose(mod, g, g)
    return fcompose(mod, f, gg) if odd else gg

def find_card(value, ncards, instructions):
    f = inverse_function(instructions, ncards)
    return next(i for i in range(ncards) if fapply(f, i, ncards) == value)

def card_at(index, ncards, nshuffles, instructions):
    f = inverse_function(instructions, ncards)
    fn = frepeat(f, nshuffles, ncards)
    return fapply(fn, index, ncards)

instructions = input_data.split("\n")
print(f"Part 2 solution: {card_at(2020, 119315717514047, 101741582076661, instructions)}")

Part 2 solution: 98461321956136
